### Adding data from FOIA requests

The data in the Chicago Data Portal only includes employee reimbursements paid after Jan. 1 of the previous calendar year. I am in the process of submitting FOIA requests to get data from further back, so that my site can host both historical and current data in one place.

In [57]:
import pandas as pd
import glob
import os

In [66]:
path = '/Users/hgorledeenn/Desktop/Chicago-Reimbursements-site/foia_data'
all_files = glob.glob(os.path.join(path, "*.xlsx"))
columns_to_use = ['Dept Code', 'Pymt Date', 'Vendor Name', 'Vendor Number', 'Voucher (Batch) Number', 'Invoice Line Nbr', 'Invoice Line Amount', 'Invoice Number', 'Invoice Date', 'Invoice Description']

In [ ]:
df_list = []

for filename in all_files:
    df_small = pd.read_excel(filename, usecols=columns_to_use)
    df_list.append(df_small)
    df = pd.concat(df_list, ignore_index=True)

df = df[df['Invoice Line Nbr'] != 'Sum:']

df.dtypes

Dept Code                        float64
Pymt Date                 datetime64[ns]
Vendor Name                       object
Vendor Number                    float64
Voucher (Batch) Number            object
Invoice Line Nbr                  object
Invoice Line Amount              float64
Invoice Number                    object
Invoice Date              datetime64[ns]
Invoice Description               object
dtype: object

In [ ]:
df.groupby('Invoice Line Nbr')['networthusbillion'].median()